# Intro

This notebook describes how to download NESDIS satellite data from the the Amazon cloud serves (AWS) using the nesdis_aws library. nesdis_aws is a wrapper around the s3fs library which is briefly intorduced further down on this page.

For info on the available data projects go to: https://docs.opendata.aws/noaa-goes16/cics-readme.html

# imports

In [3]:
# Module to interface with s3 (AWS)
import s3fs
import pathlib as pl

# The nesdis_aws package

availble here: https://github.com/hagne/nesdis_aws

In [2]:
import nesdis_aws

Initiate a data query. This will generate a list of available files. One can estimate the disk space that is needed and if sufficient space is available

In [7]:
query = nesdis_aws.AwsQuery(path2folder_local='/mnt/telg/data/smoke_events/20200912_18_CO/goes_raw/',
                            satellite='16',
                            product='ABI-L2-AOD',
                            scan_sector='C',
                            start='2020-09-12 12:00:00',
                            end='2020-09-19 13:00:00',
                            no_of_days=None,
                            last_x_days=None,
                            max_no_of_files=100)

In [8]:
print(query.info_on_current_query())


no of files: 2026
estimated disk usage: 7771 mb
remaining disk space after download: 48 %



## get subset of workplan

The workplan is a pandas DataFrame, that can be altered like truncated or resampled.

Only consider every 10th row:

In [9]:
query.workplan[::10]

,path2file_aws,path2file_local
2020-09-12 12:01:15,noaa-goes16/ABI-L2-AODC/2020/256/12/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-12 12:51:15,noaa-goes16/ABI-L2-AODC/2020/256/12/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-12 13:41:15,noaa-goes16/ABI-L2-AODC/2020/256/13/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-12 14:31:15,noaa-goes16/ABI-L2-AODC/2020/256/14/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-12 15:21:15,noaa-goes16/ABI-L2-AODC/2020/256/15/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
...,...,...
2020-09-19 09:11:16,noaa-goes16/ABI-L2-AODC/2020/263/09/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-19 10:01:16,noaa-goes16/ABI-L2-AODC/2020/263/10/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-19 10:51:16,noaa-goes16/ABI-L2-AODC/2020/263/10/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-19 11:41:16,noaa-goes16/ABI-L2-AODC/2020/263/11/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...


Only one file every 15 minutes

In [10]:
resampled = query.workplan.resample('15min').first()
resampled

,path2file_aws,path2file_local
2020-09-12 12:00:00,noaa-goes16/ABI-L2-AODC/2020/256/12/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-12 12:15:00,noaa-goes16/ABI-L2-AODC/2020/256/12/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-12 12:30:00,noaa-goes16/ABI-L2-AODC/2020/256/12/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-12 12:45:00,noaa-goes16/ABI-L2-AODC/2020/256/12/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-12 13:00:00,noaa-goes16/ABI-L2-AODC/2020/256/13/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
...,...,...
2020-09-19 11:45:00,noaa-goes16/ABI-L2-AODC/2020/263/11/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-19 12:00:00,noaa-goes16/ABI-L2-AODC/2020/263/12/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-19 12:15:00,noaa-goes16/ABI-L2-AODC/2020/263/12/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...
2020-09-19 12:30:00,noaa-goes16/ABI-L2-AODC/2020/263/12/OR_ABI-L2-...,/mnt/telg/data/smoke_events/20200912_18_CO/goe...


To update the query's workplan overwrite it with the altered one:

In [11]:
query.workplan = resampled

In [12]:
print(query.info_on_current_query())

no of files: 676
estimated disk usage: 2512 mb
remaining disk space after download: 48 %



## download the files

To download all files in the workplan execute:

In [ ]:
query.download()

# AWS and the s3fs library

This is an introduction into how to use the s3fs library for acessing the AWS system ... In case you don't want to use the nesdis_aws package or simply want to understand the key library in it.

## connect to file system

In [1]:
import s3fs

In [2]:
aws = s3fs.S3FileSystem(anon=True)

## explore file system

In [3]:
satellite = 16#16 (east) or 17(west)
base_folder = pl.Path(f'noaa-goes{satellite}')
base_folder

PosixPath('noaa-goes16')

### products

In [4]:
products_available = aws.glob(base_folder.joinpath('*').as_posix())
products_available

['noaa-goes16/ABI-L1b-RadC',
 'noaa-goes16/ABI-L1b-RadF',
 'noaa-goes16/ABI-L1b-RadM',
 'noaa-goes16/ABI-L2-ACHAC',
 'noaa-goes16/ABI-L2-ACHAF',
 'noaa-goes16/ABI-L2-ACHAM',
 'noaa-goes16/ABI-L2-ACHTF',
 'noaa-goes16/ABI-L2-ACHTM',
 'noaa-goes16/ABI-L2-ACMC',
 'noaa-goes16/ABI-L2-ACMF',
 'noaa-goes16/ABI-L2-ACMM',
 'noaa-goes16/ABI-L2-ACTPC',
 'noaa-goes16/ABI-L2-ACTPF',
 'noaa-goes16/ABI-L2-ACTPM',
 'noaa-goes16/ABI-L2-ADPC',
 'noaa-goes16/ABI-L2-ADPF',
 'noaa-goes16/ABI-L2-ADPM',
 'noaa-goes16/ABI-L2-AICEF',
 'noaa-goes16/ABI-L2-AITAF',
 'noaa-goes16/ABI-L2-AODC',
 'noaa-goes16/ABI-L2-AODF',
 'noaa-goes16/ABI-L2-CMIPC',
 'noaa-goes16/ABI-L2-CMIPF',
 'noaa-goes16/ABI-L2-CMIPM',
 'noaa-goes16/ABI-L2-CODC',
 'noaa-goes16/ABI-L2-CODF',
 'noaa-goes16/ABI-L2-CPSC',
 'noaa-goes16/ABI-L2-CPSF',
 'noaa-goes16/ABI-L2-CPSM',
 'noaa-goes16/ABI-L2-CTPC',
 'noaa-goes16/ABI-L2-CTPF',
 'noaa-goes16/ABI-L2-DMWC',
 'noaa-goes16/ABI-L2-DMWF',
 'noaa-goes16/ABI-L2-DMWM',
 'noaa-goes16/ABI-L2-DMWVC',
 'n

### example product AOD conus

In [18]:
product = 'ABI-L2-DMWVC'#'ABI-L1b-RadC'#"ABI-L2-AODC"

In [19]:
years_available = aws.glob(base_folder.joinpath(product).joinpath('*').as_posix())
years_available

['noaa-goes16/ABI-L2-DMWVC/2021']

In [20]:
year = 2021

In [21]:
days_available = aws.glob(base_folder.joinpath(product).joinpath(f'{year}').joinpath('*').as_posix())
days_available

['noaa-goes16/ABI-L2-DMWVC/2021/102',
 'noaa-goes16/ABI-L2-DMWVC/2021/103',
 'noaa-goes16/ABI-L2-DMWVC/2021/104',
 'noaa-goes16/ABI-L2-DMWVC/2021/105',
 'noaa-goes16/ABI-L2-DMWVC/2021/106',
 'noaa-goes16/ABI-L2-DMWVC/2021/107',
 'noaa-goes16/ABI-L2-DMWVC/2021/108',
 'noaa-goes16/ABI-L2-DMWVC/2021/109',
 'noaa-goes16/ABI-L2-DMWVC/2021/110',
 'noaa-goes16/ABI-L2-DMWVC/2021/111',
 'noaa-goes16/ABI-L2-DMWVC/2021/112',
 'noaa-goes16/ABI-L2-DMWVC/2021/113']

In [15]:
day = 90

In [16]:
hours_available = aws.glob(base_folder.joinpath(product).joinpath(f'{year}').joinpath(f'{day:03d}').joinpath('*').as_posix())
hours_available

['noaa-goes16/ABI-L1b-RadC/2021/090/00',
 'noaa-goes16/ABI-L1b-RadC/2021/090/01',
 'noaa-goes16/ABI-L1b-RadC/2021/090/02',
 'noaa-goes16/ABI-L1b-RadC/2021/090/03',
 'noaa-goes16/ABI-L1b-RadC/2021/090/04',
 'noaa-goes16/ABI-L1b-RadC/2021/090/05',
 'noaa-goes16/ABI-L1b-RadC/2021/090/06',
 'noaa-goes16/ABI-L1b-RadC/2021/090/07',
 'noaa-goes16/ABI-L1b-RadC/2021/090/08',
 'noaa-goes16/ABI-L1b-RadC/2021/090/09',
 'noaa-goes16/ABI-L1b-RadC/2021/090/10',
 'noaa-goes16/ABI-L1b-RadC/2021/090/11',
 'noaa-goes16/ABI-L1b-RadC/2021/090/12',
 'noaa-goes16/ABI-L1b-RadC/2021/090/13',
 'noaa-goes16/ABI-L1b-RadC/2021/090/14',
 'noaa-goes16/ABI-L1b-RadC/2021/090/15',
 'noaa-goes16/ABI-L1b-RadC/2021/090/16',
 'noaa-goes16/ABI-L1b-RadC/2021/090/17',
 'noaa-goes16/ABI-L1b-RadC/2021/090/18',
 'noaa-goes16/ABI-L1b-RadC/2021/090/19',
 'noaa-goes16/ABI-L1b-RadC/2021/090/20',
 'noaa-goes16/ABI-L1b-RadC/2021/090/21',
 'noaa-goes16/ABI-L1b-RadC/2021/090/22',
 'noaa-goes16/ABI-L1b-RadC/2021/090/23']

In [17]:
hour = 9
files_available = aws.glob(base_folder.joinpath(product).joinpath(f'{year}').joinpath(f'{day:03d}').joinpath(f'{hour:02d}').joinpath('*').as_posix())
files_available

['noaa-goes16/ABI-L1b-RadC/2021/090/09/OR_ABI-L1b-RadC-M6C01_G16_s20210900901154_e20210900903527_c20210900903571.nc',
 'noaa-goes16/ABI-L1b-RadC/2021/090/09/OR_ABI-L1b-RadC-M6C01_G16_s20210900906154_e20210900908527_c20210900908563.nc',
 'noaa-goes16/ABI-L1b-RadC/2021/090/09/OR_ABI-L1b-RadC-M6C01_G16_s20210900911154_e20210900913527_c20210900913562.nc',
 'noaa-goes16/ABI-L1b-RadC/2021/090/09/OR_ABI-L1b-RadC-M6C01_G16_s20210900916154_e20210900918527_c20210900918569.nc',
 'noaa-goes16/ABI-L1b-RadC/2021/090/09/OR_ABI-L1b-RadC-M6C01_G16_s20210900921154_e20210900923527_c20210900923566.nc',
 'noaa-goes16/ABI-L1b-RadC/2021/090/09/OR_ABI-L1b-RadC-M6C01_G16_s20210900926154_e20210900928527_c20210900928561.nc',
 'noaa-goes16/ABI-L1b-RadC/2021/090/09/OR_ABI-L1b-RadC-M6C01_G16_s20210900931154_e20210900933527_c20210900933574.nc',
 'noaa-goes16/ABI-L1b-RadC/2021/090/09/OR_ABI-L1b-RadC-M6C01_G16_s20210900936154_e20210900938527_c20210900938565.nc',
 'noaa-goes16/ABI-L1b-RadC/2021/090/09/OR_ABI-L1b-RadC-M

## Download a file

In [48]:
path2file_aws = pl.Path(files_available[3])
path2folder_loc =pl.Path('/mnt/telg/tmp/aws_tmp/') 

In [49]:
path2file_local = path2folder_loc.joinpath(path2file_aws.name)
path2file_local

PosixPath('/mnt/telg/tmp/aws_tmp/OR_ABI-L2-AODC-M6_G16_s20200900916180_e20200900918553_c20200900919522.nc')

In [51]:
aws.get(path2file_aws.as_posix(), path2file_local.as_posix())

[None]